# Adding New Model Elements

This notebook walks through adding in-memory elements via PyMBE and tracking the additions to support updating model repositories after a session.

In [ ]:
import pymbe.api as pm
from importlib import resources as lib_resources

from uuid import uuid4

from pymbe.model import Element
from pymbe.model import MetaModel
from pymbe.model_modification import \
    build_from_classifier_pattern, new_element_ownership_pattern

from pymbe.metamodel import \
    get_more_general_types, derive_inherited_featurememberships

## Open Library Data

The library data are organized around namespaces, where each namespace corresponds to a specific file in the library.

In [ ]:
library_model = None

with lib_resources.path("pymbe.static_data", "KernelLibrary.json") as lib_data:
    library_model = pm.Model.load_from_post_file(lib_data)

In [ ]:
library_model.ownedElement[0]._data

## New Package
The most basic element in a model for grouping things is the package. It holds a place within a modeling project to associate other model contents.

The new root-level Package also needs to be inside a Namespace and become an owned member via an OwningMembership.

In [ ]:
package_model_namespace_data = {
    'aliasIds': [],
    'isImpliedIncluded': False,
    '@type': "Namespace",
    '@id': str(uuid4()),
    'ownedRelationship': []
}

In [ ]:
package_model_data = {
    'name': "User Process Model",
    'isLibraryElement': False,
    'filterCondition': [],
    'ownedElement': [],
    'owner': {},
    '@type': "Package",
    '@id': str(uuid4()),
    'ownedRelationship': []
}

In [ ]:
empty_model = pm.Model(elements={})
empty_model

In [ ]:
new_ns = Element.new(data=package_model_namespace_data,model=empty_model)

In [ ]:
new_package = Element.new(data=package_model_data,model=empty_model)
new_package

In [ ]:
new_element_ownership_pattern(
    owner=new_ns, ele=new_package, model=empty_model, member_kind="OwningMembership"
)

### Connect New Model to Library

Reference the new model to the library so inheritance and redefinition can be connected to library elements.

In [ ]:
empty_model.reference_other_model(library_model)

In [ ]:
peform_ns = [library_model_ns
               for library_model_ns in library_model.ownedElement
               if library_model_ns.throughOwningMembership[0].declaredName == 'Performances'][0]

perform_eles = peform_ns.throughOwningMembership[0].throughOwningMembership

performance = None
    
for perform_ele in perform_eles:
    if perform_ele._metatype in ('Behavior'):
        if hasattr(perform_ele, "declaredName"):
            if perform_ele.declaredName == 'Performance':
                performance = perform_ele
                
performance

In [ ]:
performance._metamodel_hints

In [ ]:
performance._data

## New Classifier

For the KerML side of the metamodel, Classifier is one of the most basic kinds. We can build up a dictionary with owned attributes.

In [ ]:
new_performance = build_from_classifier_pattern(
    owner=new_package,
    name="New Process",
    model=empty_model,
    metatype="Behavior",
    superclasses=[performance],
    specific_fields={"ownedRelationship": []}
)

In [ ]:
new_package.throughOwningMembership

In [ ]:
new_process = new_package.throughOwningMembership[0]
new_process

In [ ]:
new_process._data

In [ ]:
new_process._derived

In [ ]:
get_more_general_types(new_process,0,20)

In [ ]:
[feature_membership.target[0]
        for feature_membership in derive_inherited_featurememberships(new_process)
    ] + new_process.throughFeatureMembership

In [ ]:
new_process._data

In [ ]:
new_process.feature

In [ ]:
performance.ownedRelationship

In [ ]:
process_model_package_data = {
    'name': "Battery Pack",
    'isLibraryElement': False,
    'documentation': [],
    'isConjugated': False,
    'isSufficient': False,
    'isAbstract': False,
    'ownedElement': [],
    'owner': {},
    '@type': "Classifier",
    '@id': str(uuid4())
}
new_classifier = Element.new(data=classifier_model_data,model=sysml_model)
new_classifier

In [ ]:
process_model_package_data = {
    'name': "Battery Pack",
    'isLibraryElement': False,
    'documentation': [],
    'isConjugated': False,
    'isSufficient': False,
    'isAbstract': False,
    'ownedElement': [],
    'owner': {},
    '@type': "Classifier",
    '@id': str(uuid4())
}
new_classifier = Element.new(data=classifier_model_data,model=sysml_model)
new_classifier

## New Feature
A new Feature has some owned attributes and can be standalone in KerML.

## New OwningMembership

The OwningMembership, of which the OwningFeatureMembership is a sub-metatype, is what is used to organize the model hierarchy. Connecting two elements with this relationship should also updated derived fields such as owner and ownedElements.

In [ ]:
new_om_source = new_package
new_om_target = new_classifier
new_om_source_id = new_package._id
new_om_target_id = new_classifier._id
om_data = {
    'isLibraryElement': False,
    'documentation': [],
    'source': [{'@id': new_om_source_id}],
    'target': [{'@id': new_om_target_id}],
    'owningRelatedElement': {'@id': new_om_source_id},
    'ownedRelatedElement': [{'@id': new_om_target_id}],
    'relatedElement': [{'@id': new_om_source_id}, {'@id': new_om_target_id}],
    '@type': "OwningMembership",
    '@id': str(uuid4())
}
new_om = Element.new(data=om_data,model=sysml_model)
new_om
(new_om_source_id, new_om_target_id)


In [ ]:
new_om

In [ ]:
new_package.ownedElement

### Derive Properties Due to Relationship

On the two ends of the relationship are fields that need to have fields updated

In [ ]:
new_package._data.update(
    {'ownedElement': new_package._data['ownedElement'] + [{'@id': new_om_target_id}]
    }
)
new_package.resolve()

new_classifier._data.update(
    {'owner': {'@id': new_om_source_id}
    }
)
new_classifier.resolve()

In [ ]:
new_package._data

In [ ]:
new_package.ownedElement

In [ ]:
new_classifier.owner

In [ ]:
new_package['ownedElement']